In [2]:
import sys
import os

#####################
# Import of utils.py functions
#####################
# Required to get utils.py and access its functions
notebook_dir = os.path.abspath("")
parent_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
sys.path.append(parent_dir)
sys.path.append('.')
from utils import loadFSL, FSLeyesServer, mkdir_no_exist

#############################
# Loading fsl and freesurfer within Neurodesk
# You can find the list of available other modules by clicking on the "Softwares" tab on the left
#############################
import lmod
await lmod.purge(force=True)
await lmod.load('fsl/6.0.7.4')
await lmod.load('freesurfer/7.4.1')
await lmod.list()

####################
# Setup FSL path
####################
loadFSL()

####################
# DIPY_HOME should be set prior to import of dipy to make sure all downloads point to the right folder
####################
os.environ["DIPY_HOME"] = "/home/jovyan/Data"

###################
# Load other relevant libraries
##################
import fsl.wrappers, statsmodels
from fsl.wrappers import fslmaths
import dipy
import os.path as op
import nibabel as nib
from mne_bids import BIDSPath, read_raw_bids, print_dir_tree, make_report
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tools.sm_exceptions import ConvergenceWarning
from nilearn import plotting
from sklearn.decomposition import FastICA

In [7]:
# Loading data, preprocessed data (smoothed) as fmri_img

In [3]:
# organisation of directory and creation of preprocessing folders
dataset_id = 'subject101410'
subject = '101410' 

sample_path = "/home/jovyan/data"
mkdir_no_exist(sample_path)
bids_root = op.join(sample_path, dataset_id)
deriv_root = op.join(bids_root, 'derivatives')
preproc_root = op.join(bids_root, 'derivatives','preprocessed_data')

mkdir_no_exist(bids_root)

subject_dir = 'sub-{}'.format(subject)



###################
# Create folders relevant for preprocessing.
# In BIDs, ANYTHING we modify must go in the derivatives folder, to keep original files clean in case we make a mistake.
###################
mkdir_no_exist(op.join(bids_root, 'derivatives'))
preproc_root = op.join(bids_root, 'derivatives','preprocessed_data')
mkdir_no_exist(preproc_root)
mkdir_no_exist(op.join(preproc_root, 'sub-101410'))
mkdir_no_exist(op.join(preproc_root, 'sub-101410', 'anat'))
mkdir_no_exist(op.join(preproc_root, 'sub-101410', 'func'))
mkdir_no_exist(op.join(preproc_root, 'sub-101410', 'fmap')) 

In [4]:
# get preprocessed fMRI image

smoothed_path = op.join(preproc_root, "sub-101410", "func", "sub-101410_task-motor_concat_varnorm_moco_smoothed-6mm.nii.gz")
fmri_img = nib.load(smoothed_path)

In [ ]:
# Apply ICA on the subject fMRI runs, considering one volume as one sample.

In [5]:
# --- Get data as NumPy array ---
data = fmri_img.get_fdata()  # shape: (nx, ny, nz, nTRs)
nx, ny, nz, nTRs = data.shape
print(f"fMRI data shape: {data.shape} (x,y,z,time)")

fMRI data shape: (91, 109, 91, 568) (x,y,z,time)


In [6]:
# --- Reshape for spatial ICA ---
# Each volume = one sample, each voxel = one feature
X = data.reshape(-1, nTRs).T  # shape (nTRs, nx*ny*nz)
print(f"X shape for ICA (samples x features): {X.shape}")

X shape for ICA (samples x features): (568, 902629)


In [7]:
# --- Set number of components ---
n_components = 20  # justify in your report (10-30 typical for single-subject fMRI)

In [ ]:
# --- Run spatial ICA ---
ica = FastICA(n_components=n_components, random_state=0)
S = ica.fit_transform(X)       # shape: (nTRs, n_components) → timecourses
A = ica.mixing_                # shape: (n_voxels, n_components) → spatial maps
print(f"S (timecourses) shape: {S.shape}")
print(f"A (spatial maps) shape: {A.shape}")